In [ ]:
input_test = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""

In [ ]:
import numpy as np

In [ ]:
udlr = {"u" : np.array((-1,0)), "d" : np.array((1,0)), "l" : np.array((0,-1)), "r" : np.array((0,1)) }

def parse_input(inp):
    g = np.array([[c for c in line] for line in inp.split('\n')])
    #slice out framing walls: slightly simplifies things, never beneficial to enter them
    g = g[1:g.shape[0]-1,1:g.shape[1]-1]
    p_start = np.argwhere(g == "S")[0]
    p_end = np.argwhere(g == "E")[0]
    g[tuple(p_start)] = "."
    g[tuple(p_end)] = "."
    return g, p_start, p_end

def p_in_g(p, g):
    if np.any(p < (0,0)) or np.any(p >= g.shape): return False
    return True

def solve(inp, cutoff, duration=20):
    vmax = 1e9
    g, p_start, p_end = parse_input(inp)
    g_start_distance = np.zeros(g.shape, dtype=np.int64) + vmax
    g_end_distance = np.zeros(g.shape, dtype=np.int64) + vmax
    
    for p_from, g_dist in zip([p_start, p_end], [g_start_distance, g_end_distance]):
        g_dist[tuple(p_from)] = 0
        queue = [p_from]
        
        while queue:
            p = queue.pop(0)
            for d in udlr.values():
                P = p + d
                if not p_in_g(P, g): continue
                if g[tuple(P)] == "#": continue
                if g_dist[tuple(P)] != vmax: continue
                g_dist[tuple(P)] = g_dist[tuple(p)] + 1
                queue.append(P)
    
    distance_no_cheat = g_start_distance[tuple(p_end)]
    list_start = sorted([(g_start_distance[p], p) for p in np.ndindex(g.shape) if g_start_distance[p] < distance_no_cheat], key=lambda x: x[0])
    list_end = sorted([(g_end_distance[p], p) for p in np.ndindex(g.shape) if g_end_distance[p] < distance_no_cheat], key=lambda x: x[0])

    tot = 0    
    for c_start,p_start in list_start:
        for c_end,p_end in list_end:
            if c_start + c_end + cutoff > distance_no_cheat: break
            distance = sum([abs(p_start[i] - p_end[i]) for i in range(2)])
            if distance > duration: continue
            cost_save_cheat = distance_no_cheat - distance - c_start - c_end
            if cost_save_cheat >= cutoff: tot += 1
    return tot


In [ ]:
assert solve(input_test, 12, 2) == 8
#p1 solution is inefficient but can reuse p2 solution
solve(input_full, 100, 2)

In [ ]:
solve(input_full, 100, 20)

In [ ]:
input_full = """#############################################################################################################################################
#...#...#...#...#...#.....###...###...#...#.....#...#...#...###...#...........#...###.....#...#.........#...#.....#.....#...#.....###...#...#
#.#.#.#.#.#.#.#.#.#.#.###.###.#.###.#.#.#.#.###.#.#.#.#.#.#.###.#.#.#########.#.#.###.###.#.#.#.#######.#.#.#.###.#.###.#.#.#.###.###.#.#.#.#
#.#.#.#.#.#.#.#.#.#.#.#...#...#.....#.#.#.#...#...#.#.#.#.#...#.#.#.#.........#.#...#.#...#.#...#.......#.#.#.#...#...#.#.#.#...#.....#...#.#
#.#.#.#.#.#.#.#.#.#.#.#.###.#########.#.#.###.#####.#.#.#.###.#.#.#.#.#########.###.#.#.###.#####.#######.#.#.#.#####.#.#.#.###.###########.#
#.#...#...#.#.#...#.#.#...#.........#.#.#...#.....#.#.#.#.#...#.#.#.#.###...###.#...#.#...#.#.....#.....#.#.#.#.#####.#.#.#...#.#...........#
#.#########.#.#####.#.###.#########.#.#.###.#####.#.#.#.#.#.###.#.#.#.###.#.###.#.###.###.#.#.#####.###.#.#.#.#.#####.#.#.###.#.#.###########
#.#...#...#.#.#...#.#.#...#...#...#.#.#.#...###...#.#.#...#.#...#.#.#...#.#.#...#...#...#.#.#.#...#...#.#.#.#.#...#...#.#.#...#.#.###...#...#
#.#.#.#.#.#.#.#.#.#.#.#.###.#.#.#.#.#.#.#.#####.###.#.#####.#.###.#.###.#.#.#.#####.###.#.#.#.#.#.###.#.#.#.#.###.#.###.#.#.###.#.###.#.#.#.#
#.#.#...#.#.#.#.#...#.#.#...#.#.#.#.#...#.#...#...#.#.....#.#...#...#...#.#...#.....#...#.#.#.#.#.#...#...#.#...#.#...#.#.#.#...#...#.#.#.#.#
#.#.#####.#.#.#.#####.#.#.###.#.#.#.#####.#.#.###.#.#####.#.###.#####.###.#####.#####.###.#.#.#.#.#.#######.###.#.###.#.#.#.#.#####.#.#.#.#.#
#.#.#.....#...#...#...#.#...#.#.#.#.#.....#.#.#...#...#...#...#.....#...#.....#.#...#...#...#.#.#...#.......###.#.#...#...#...#...#...#...#.#
#.#.#.###########.#.###.###.#.#.#.#.#.#####.#.#.#####.#.#####.#####.###.#####.#.#.#.###.#####.#.#####.#########.#.#.###########.#.#########.#
#...#...........#.#.#...#...#...#...#...###.#.#...#...#.....#.#.....#...#.....#.#.#.....#.....#.....#.......#...#.#.#...#...#...#...........#
###############.#.#.#.###.#############.###.#.###.#.#######.#.#.#####.###.#####.#.#######.#########.#######.#.###.#.#.#.#.#.#.###############
#...###.........#.#.#...#.......#.....#.....#.#...#.....#...#.#.....#.###.....#...#.......#...#...#.#.......#...#.#.#.#...#.#...###...#...###
#.#.###.#########.#.###.#######.#.###.#######.#.#######.#.###.#####.#.#######.#####.#######.#.#.#.#.#.#########.#.#.#.#####.###.###.#.#.#.###
#.#...#.........#.#...#.#.....#...###.......#.#.#.......#...#.....#.#...#...#.....#.......#.#.#.#.#.#...#.....#.#...#.....#.....#...#...#...#
#.###.#########.#.###.#.#.###.#############.#.#.#.#########.#####.#.###.#.#.#####.#######.#.#.#.#.#.###.#.###.#.#########.#######.#########.#
#...#.#...#...#.#.###.#.#...#.#...###...#...#...#...#...###.....#...#...#.#...#...#.......#.#...#.#...#.#...#.#...#.......#...#...#.........#
###.#.#.#.#.#.#.#.###.#.###.#.#.#.###.#.#.#########.#.#.#######.#####.###.###.#.###.#######.#####.###.#.###.#.###.#.#######.#.#.###.#########
#...#...#.#.#...#...#.#.#...#.#.#.#...#.#.......#...#.#...#.....#.....###...#...#...#...###.#.....###.#.###.#.#...#.#...###.#.#.#...#...#...#
#.#######.#.#######.#.#.#.###.#.#.#.###.#######.#.###.###.#.#####.#########.#####.###.#.###.#.#######.#.###.#.#.###.#.#.###.#.#.#.###.#.#.#.#
#.#.....#.#.......#...#.#...#.#.#.#.#...#.......#...#.#...#.....#.#.....###...#...#...#.....#.#.....#.#.#...#.#...#...#.#...#...#.....#...#.#
#.#.###.#.#######.#####.###.#.#.#.#.#.###.#########.#.#.#######.#.#.###.#####.#.###.#########.#.###.#.#.#.###.###.#####.#.#################.#
#.#.#...#.....###.....#.###.#.#.#.#.#.###.......#...#.#...#.....#.#.#...#...#.#...#.....#.....#...#.#.#.#...#.#...#.....#.#.........#...#...#
#.#.#.#######.#######.#.###.#.#.#.#.#.#########.#.###.###.#.#####.#.#.###.#.#.###.#####.#.#######.#.#.#.###.#.#.###.#####.#.#######.#.#.#.###
#.#.#.......#.#.....#.#.#...#.#.#.#.#.#...###...#.###.#...#.....#...#...#.#.#.#...#...#.#.#...#...#...#...#.#...#...#.....#.......#.#.#.#...#
#.#.#######.#.#.###.#.#.#.###.#.#.#.#.#.#.###.###.###.#.#######.#######.#.#.#.#.###.#.#.#.#.#.#.#########.#.#####.###.###########.#.#.#.###.#
#.#.###...#.#.#.#...#.#...#...#.#.#.#.#.#.#...#...#...#.#...#...#.......#.#.#.#.###.#.#.#.#.#.#.........#...#...#.....#.....#.....#...#...#.#
#.#.###.#.#.#.#.#.###.#####.###.#.#.#.#.#.#.###.###.###.#.#.#.###.#######.#.#.#.###.#.#.#.#.#.#########.#####.#.#######.###.#.###########.#.#
#.#.#...#...#...#...#.....#.#...#.#.#.#.#.#...#...#.#...#.#...#...#.....#.#.#.#...#.#.#.#.#.#.#.....#...#.....#...#.....###.#...........#...#
#.#.#.#############.#####.#.#.###.#.#.#.#.###.###.#.#.###.#####.###.###.#.#.#.###.#.#.#.#.#.#.#.###.#.###.#######.#.#######.###########.#####
#.#.#...#.......#...#...#.#.#...#.#.#.#.#.###...#.#.#.###.....#.###...#.#.#.#.#...#.#.#.#.#.#.#...#.#...#...#...#...#.....#.#...#.......#...#
#.#.###.#.#####.#.###.#.#.#.###.#.#.#.#.#.#####.#.#.#.#######.#.#####.#.#.#.#.#.###.#.#.#.#.#.###.#.###.###.#.#.#####.###.#.#.#.#.#######.#.#
#.#.###...#.....#.....#...#.....#...#.#.#...#...#.#.#.#...#...#.#...#.#.#.#...#...#.#.#.#.#.#...#.#.#...#...#.#.#...#...#.#...#...#.......#.#
#.#.#######.#########################.#.###.#.###.#.#.#.#.#.###.#.#.#.#.#.#######.#.#.#.#.#.###.#.#.#.###.###.#.#.#.###.#.#########.#######.#
#.#.#.......#.....#...#...#.....#...#...###.#.###.#.#.#.#...###.#.#.#.#.#.....#...#.#...#.#.#...#.#.#.###.....#...#.#...#...#.....#...#.....#
#.#.#.#######.###.#.#.#.#.#.###.#.#.#######.#.###.#.#.#.#######.#.#.#.#.#####.#.###.#####.#.#.###.#.#.#############.#.#####.#.###.###.#.#####
#...#.........###...#...#...###...#.......#.#...#.#.#.#.....#...#.#...#.#...#.#...#...#...#.#.....#...#.............#.#.....#.#...#...#.....#
#########################################.#.###.#.#.#.#####.#.###.#####.#.#.#.###.###.#.###.###########.#############.#.#####.#.###.#######.#
#.........................#...............#.....#...#.#.....#...#.....#.#.#.#.#...#...#...#...#.......#...............#...#...#...#.#.......#
#.#######################.#.#########################.#.#######.#####.#.#.#.#.#.###.#####.###.#.#####.###################.#.#####.#.#.#######
#.......#...#...#.......#.#.#.................#.......#.......#.#...#.#.#.#.#.#...#.#...#.#...#.#...#...........#...#...#...#...#...#.......#
#######.#.#.#.#.#.#####.#.#.#.###############.#.#############.#.#.#.#.#.#.#.#.###.#.#.#.#.#.###.#.#.###########.#.#.#.#.#####.#.###########.#
#...###...#...#.#.#...#.#.#...#.........#.....#.....#.........#.#.#...#.#.#...#...#...#.#.#...#...#.......#...#...#.#.#.#.....#...........#.#
#.#.###########.#.#.#.#.#.#####.#######.#.#########.#.#########.#.#####.#.#####.#######.#.###.###########.#.#.#####.#.#.#.###############.#.#
#.#.#...#.......#.#.#...#.......#.....#.#.....#.....#.......#...#.#.....#...#...#.....#.#.#...#.....#...#...#.....#.#.#.#.............#...#.#
#.#.#.#.#.#######.#.#############.###.#.#####.#.###########.#.###.#.#######.#.###.###.#.#.#.###.###.#.#.#########.#.#.#.#############.#.###.#
#.#...#.#.........#.#...#.........#...#.......#.....#...#...#.#...#...###E#.#...#...#.#.#...###...#.#.#.....#.....#...#.........#...#.#...#.#
#.#####.###########.#.#.#.#########.###############.#.#.#.###.#.#####.###.#.###.###.#.#.#########.#.#.#####.#.#################.#.#.#.###.#.#
#...#...#...#.......#.#.#.#.........#...#...#...#...#.#.#...#.#.#.....###.#.#...#...#...#...#...#.#...#...#...#...#...#.......#.#.#...###...#
###.#.###.#.#.#######.#.#.#.#########.#.#.#.#.#.#.###.#.###.#.#.#.#######.#.#.###.#######.#.#.#.#.#####.#.#####.#.#.#.#.#####.#.#.###########
#...#.#...#...#.......#.#.#...........#...#...#.#...#.#.#...#.#.#.#...###.#.#...#.#.......#...#...#.....#.......#...#...#.....#.#...........#
#.###.#.#######.#######.#.#####################.###.#.#.#.###.#.#.#.#.###.#.###.#.#.###############.#####################.#####.###########.#
#...#.#.........#...###...#.......#...#.........###.#.#.#.###.#.#...#.###...###...#...#.......#.....#.......#.............###...#.....#.....#
###.#.###########.#.#######.#####.#.#.#.###########.#.#.#.###.#.#####.###############.#.#####.#.#####.#####.#.###############.###.###.#.#####
#...#.#.........#.#.#...###.....#.#.#.#...........#.#.#.#...#.#.#.....#########...###...#...#...#...#.....#...#...###...#...#.....###...#...#
#.###.#.#######.#.#.#.#.#######.#.#.#.###########.#.#.#.###.#.#.#.#############.#.#######.#.#####.#.#####.#####.#.###.#.#.#.#############.#.#
#...#.#...#...#...#.#.#.#.....#.#...#.........#...#...#.#...#.#.#.###########...#...#.....#.......#.....#.......#.....#...#...............#.#
###.#.###.#.#.#####.#.#.#.###.#.#############.#.#######.#.###.#.#.###########.#####.#.#################.###################################.#
#...#...#...#.....#.#.#.#...#.#.............#.#.###...#...#...#.#.###########.....#.#.......#.........#.#.........#.........................#
#.#####.#########.#.#.#.###.#.#############.#.#.###.#.#####.###.#.###############.#.#######.#.#######.#.#.#######.#.#########################
#.#...#...#.......#.#.#.#...#...........#...#.#.....#.....#...#.#.###########.....#...#.....#.###.....#...#...###...#.......#...#...###...###
#.#.#.###.#.#######.#.#.#.#############.#.###.###########.###.#.#.###########.#######.#.#####.###.#########.#.#######.#####.#.#.#.#.###.#.###
#...#...#.#.......#...#...#...........#.#...#.#.....#...#...#.#.#.###########.....#...#.....#...#.#...#...#.#.#.....#.#...#...#...#.....#...#
#######.#.#######.#########.#########.#.###.#.#.###.#.#.###.#.#.#.###############.#.#######.###.#.#.#.#.#.#.#.#.###.#.#.#.#################.#
#...#...#.........#...#...#.#.........#.....#.#...#.#.#.#...#.#.#.#####...........#.....###.....#.#.#...#...#.#.#...#.#.#...........#...#...#
#.#.#.#############.#.#.#.#.#.###############.###.#.#.#.#.###.#.#.#####.###############.#########.#.#########.#.#.###.#.###########.#.#.#.###
#.#.#.....#...#.....#.#.#.#.#.#.............#.#...#...#...###.#.#S#####...............#...#.....#...#...#.....#.#.....#...........#.#.#.#...#
#.#.#####.#.#.#.#####.#.#.#.#.#.###########.#.#.#############.#.#####################.###.#.###.#####.#.#.#####.#################.#.#.#.###.#
#.#.......#.#.#.#.....#.#.#.#.#.#...........#...#...#...#...#...#.....###...###...###...#.#...#.#...#.#...#...#.#.............#...#.#.#...#.#
#.#########.#.#.#.#####.#.#.#.#.#.###############.#.#.#.#.#.#####.###.###.#.###.#.#####.#.###.#.#.#.#.#####.#.#.#.###########.#.###.#.###.#.#
#.........#.#...#.......#...#...#.................#.#.#...#...###...#.#...#.#...#.......#.#...#...#...#.....#...#.#.........#...###...###...#
#########.#.#######################################.#.#######.#####.#.#.###.#.###########.#.###########.#########.#.#######.#################
#...###...#.#...............#...........#...........#.#.......#...#.#.#...#.#.....#.....#.#.............#...#...#...#...#...###...#.........#
#.#.###.###.#.#############.#.#########.#.###########.#.#######.#.#.#.###.#.#####.#.###.#.###############.#.#.#.#####.#.#.#####.#.#.#######.#
#.#.....#...#.....#...#...#...#.....###...#.........#.#.....###.#.#.#.....#...###.#...#.#.......#.........#...#.#.....#...#...#.#.#...#...#.#
#.#######.#######.#.#.#.#.#####.###.#######.#######.#.#####.###.#.#.#########.###.###.#.#######.#.#############.#.#########.#.#.#.###.#.#.#.#
#.#...#...#.......#.#...#.......#...#.......#.....#...#.....#...#...#...#...#.#...#...#...#...#...#...#.....#...#...........#...#.###.#.#...#
#.#.#.#.###.#######.#############.###.#######.###.#####.#####.#######.#.#.#.#.#.###.#####.#.#.#####.#.#.###.#.###################.###.#.#####
#.#.#...###.........#.....#...###...#.#.......###.......#...#.........#...#.#.#.#...#...#...#.#...#.#...#...#.#.....#.......#...#.....#.....#
#.#.#################.###.#.#.#####.#.#.#################.#.###############.#.#.#.###.#.#####.#.#.#.#####.###.#.###.#.#####.#.#.###########.#
#...#.........###...#...#...#.....#...#.............#.....#.#...#...###...#.#.#...###.#...#...#.#.#.#.....###...#...#...###...#.....#.......#
#####.#######.###.#.###.#########.#################.#.#####.#.#.#.#.###.#.#.#.#######.###.#.###.#.#.#.###########.#####.###########.#.#######
#...#.......#.....#.#...#...#...#.#...#...#.........#.#.....#.#.#.#...#.#...#.......#.#...#.....#...#.........###.......#...###...#...###...#
#.#.#######.#######.#.###.#.#.#.#.#.#.#.#.#.#########.#.#####.#.#.###.#.###########.#.#.#####################.###########.#.###.#.#######.#.#
#.#.#...###.......#.#.....#...#.#...#...#...#...#...#.#.#...#.#.#...#.#...#.........#.#.#...#...............#.......#...#.#.#...#...#...#.#.#
#.#.#.#.#########.#.###########.#############.#.#.#.#.#.#.#.#.#.###.#.###.#.#########.#.#.#.#.#############.#######.#.#.#.#.#.#####.#.#.#.#.#
#.#.#.#.#...#...#.#...#.....###...........###.#.#.#.#.#.#.#.#.#...#.#.###.#.....#...#.#.#.#.#.............#.........#.#.#.#.#.#.....#.#...#.#
#.#.#.#.#.#.#.#.#.###.#.###.#############.###.#.#.#.#.#.#.#.#.###.#.#.###.#####.#.#.#.#.#.#.#############.###########.#.#.#.#.#.#####.#####.#
#.#.#.#.#.#.#.#...#...#...#.#.............#...#...#.#.#...#.#.#...#.#.#...#.....#.#.#.#...#...........#...#...#...###.#.#.#...#.......#.....#
#.#.#.#.#.#.#.#####.#####.#.#.#############.#######.#.#####.#.#.###.#.#.###.#####.#.#.###############.#.###.#.#.#.###.#.#.#############.#####
#.#.#.#.#.#.#.....#.#...#.#...#...........#...#...#.#.#...#...#...#.#.#...#.#...#.#.#.#...#.........#.#.....#...#.....#...#...#.........#...#
#.#.#.#.#.#.#####.#.#.#.#.#####.#########.###.#.#.#.#.#.#.#######.#.#.###.#.#.#.#.#.#.#.#.#.#######.#.#####################.#.#.#########.#.#
#.#...#...#.....#.#...#...#...#.###.......#...#.#...#...#.#...#...#.#.#...#.#.#.#.#.#...#...#.....#.#.......................#.#.#.......#.#.#
#.#############.#.#########.#.#.###.#######.###.#########.#.#.#.###.#.#.###.#.#.#.#.#########.###.#.#########################.#.#.#####.#.#.#
#.#...#...#...#...#...#...#.#.#.#...#...###...#...#.....#.#.#...#...#.#...#.#.#...#...#.....#...#.#...............#.........#.#...#.....#.#.#
#.#.#.#.#.#.#.#####.#.#.#.#.#.#.#.###.#.#####.###.#.###.#.#.#####.###.###.#.#.#######.#.###.###.#.###############.#.#######.#.#####.#####.#.#
#.#.#.#.#...#.....#.#.#.#.#.#...#.....#.#.....#...#...#.#.#.....#...#.#...#.#.#.......#...#.....#.#.........#...#...#...#...#.#...#.......#.#
#.#.#.#.#########.#.#.#.#.#.###########.#.#####.#####.#.#.#####.###.#.#.###.#.#.#########.#######.#.#######.#.#.#####.#.#.###.#.#.#########.#
#.#.#...#...#.....#.#...#...###.........#...#...###...#.#.#.....#...#.#.#...#.#...#.....#.......#.#.......#...#.......#...#...#.#.#.....#...#
#.#.#####.#.#.#####.###########.###########.#.#####.###.#.#.#####.###.#.#.###.###.#.###.#######.#.#######.#################.###.#.#.###.#.###
#...###...#...#...#.#...........#...#...###.#.#...#...#.#.#.#...#...#.#.#.###.#...#.#...#...###.#.#...###.................#.#...#.#.###.#...#
#######.#######.#.#.#.###########.#.#.#.###.#.#.#.###.#.#.#.#.#.###.#.#.#.###.#.###.#.###.#.###.#.#.#.###################.#.#.###.#.###.###.#
#.......#...#...#...#...#...#.....#.#.#.#...#.#.#.....#.#.#.#.#.#...#.#.#.#...#.#...#...#.#.#...#...#.#.......#...........#...###.#...#.#...#
#.#######.#.#.#########.#.#.#.#####.#.#.#.###.#.#######.#.#.#.#.#.###.#.#.#.###.#.#####.#.#.#.#######.#.#####.#.#################.###.#.#.###
#.#.....#.#.#.#.......#...#...#.....#.#.#.#...#...###...#.#.#.#.#.#...#.#.#...#.#.....#.#.#.#.#.......#.....#.#.............#...#.....#.#...#
#.#.###.#.#.#.#.#####.#########.#####.#.#.#.#####.###.###.#.#.#.#.#.###.#.###.#.#####.#.#.#.#.#.###########.#.#############.#.#.#######.###.#
#...#...#.#...#.....#.........#...#...#.#.#.#...#...#...#.#.#.#...#...#.#.#...#...#...#.#.#...#.#.....#.....#.#...#.......#...#.###...#.....#
#####.###.#########.#########.###.#.###.#.#.#.#.###.###.#.#.#.#######.#.#.#.#####.#.###.#.#####.#.###.#.#####.#.#.#.#####.#####.###.#.#######
###...#...###...###.....#...#.....#.###...#.#.#...#...#.#.#.#...#.....#.#.#.###...#.#...#.#.....#.#...#.....#.#.#.#...###.......#...#...#...#
###.###.#####.#.#######.#.#.#######.#######.#.###.###.#.#.#.###.#.#####.#.#.###.###.#.###.#.#####.#.#######.#.#.#.###.###########.#####.#.#.#
#...#...#...#.#.#.......#.#...#...#.......#.#.#...#...#.#.#.#...#.#...#.#.#.#...#...#.#...#.#...#.#...#.....#.#.#...#.............#...#...#.#
#.###.###.#.#.#.#.#######.###.#.#.#######.#.#.#.###.###.#.#.#.###.#.#.#.#.#.#.###.###.#.###.#.#.#.###.#.#####.#.###.###############.#.#####.#
#...#.#...#...#...#...#...###.#.#.#...#...#.#.#...#...#.#.#.#.###.#.#.#.#.#.#...#.#...#.###...#.#.#...#.....#.#...#...#...#.....#...#.#.....#
###.#.#.###########.#.#.#####.#.#.#.#.#.###.#.###.###.#.#.#.#.###.#.#.#.#.#.###.#.#.###.#######.#.#.#######.#.###.###.#.#.#.###.#.###.#.#####
#...#.#.#...#...#...#.#.#.....#.#.#.#...#...#.#...#...#.#.#.#...#.#.#.#.#.#.#...#.#...#.......#.#.#.#...#...#.....###...#.#...#.#.###.#.....#
#.###.#.#.#.#.#.#.###.#.#.#####.#.#.#####.###.#.###.###.#.#.###.#.#.#.#.#.#.#.###.###.#######.#.#.#.#.#.#.###############.###.#.#.###.#####.#
#...#.#...#.#.#.#...#...#.#...#.#.#.....#.....#...#...#.#.#.#...#...#...#...#.#...#...#.......#...#...#.#.........#.....#.#...#.#...#.#.....#
###.#.#####.#.#.###.#####.#.#.#.#.#####.#########.###.#.#.#.#.###############.#.###.###.###############.#########.#.###.#.#.###.###.#.#.#####
#...#.#.....#.#.#...#.....#.#.#.#.#...#.......#...#...#.#.#.#.#.......#.......#...#...#.....###...###...#.........#.#...#.#...#.#...#.#.....#
#.###.#.#####.#.#.###.#####.#.#.#.#.#.#######.#.###.###.#.#.#.#.#####.#.#########.###.#####.###.#.###.###.#########.#.###.###.#.#.###.#####.#
#.....#.....#.#.#.#...#.....#.#.#.#.#.#.......#...#...#.#.#.#...#.....#.....#...#.###.....#.....#...#...#...#.......#.###.#...#...###.#...#.#
###########.#.#.#.#.###.#####.#.#.#.#.#.#########.###.#.#.#.#####.#########.#.#.#.#######.#########.###.###.#.#######.###.#.#########.#.#.#.#
#.......#...#.#.#.#...#.#.....#.#.#.#.#.........#.#...#.#.#.....#.....###...#.#.#.#.......#.....#...###...#.#.#.....#...#...#.........#.#...#
#.#####.#.###.#.#.###.#.#.#####.#.#.#.#########.#.#.###.#.#####.#####.###.###.#.#.#.#######.###.#.#######.#.#.#.###.###.#####.#########.#####
#.....#.#.....#...#...#.#...#...#...#...#.....#.#.#...#.#.....#.....#.#...#...#.#.#...#...#.###.#...#.....#...#...#...#...###.........#.....#
#####.#.###########.###.###.#.#########.#.###.#.#.###.#.#####.#####.#.#.###.###.#.###.#.#.#.###.###.#.###########.###.###.###########.#####.#
###...#...#...###...#...###.#.#.........#.###...#...#.#...#...###...#.#...#...#.#.#...#.#.#.#...#...#.....#.......###...#...###...###.#.....#
###.#####.#.#.###.###.#####.#.#.#########.#########.#.###.#.#####.###.###.###.#.#.#.###.#.#.#.###.#######.#.###########.###.###.#.###.#.#####
#...#...#...#.....#...#.....#.#.........#.......#...#...#.#.....#...#...#.#...#.#.#...#.#...#.#...#.....#.#...........#...#.....#...#.#.....#
#.###.#.###########.###.#####.#########.#######.#.#####.#.#####.###.###.#.#.###.#.###.#.#####.#.###.###.#.###########.###.#########.#.#####.#
#.....#...#.....#...###...#...#.........#.....#.#.#.....#.....#...#...#.#.#.###...#...#.....#.#...#...#...#.....#...#...#.#.......#.#.#...#.#
#########.#.###.#.#######.#.###.#########.###.#.#.#.#########.###.###.#.#.#.#######.#######.#.###.###.#####.###.#.#.###.#.#.#####.#.#.#.#.#.#
#.........#...#.#.#.......#.###.........#...#.#.#.#.........#.....###.#.#...#...###...#.....#.#...#...#...#...#.#.#.#...#.#.....#.#.#...#.#.#
#.###########.#.#.#.#######.###########.###.#.#.#.#########.#########.#.#####.#.#####.#.#####.#.###.###.#.###.#.#.#.#.###.#####.#.#.#####.#.#
#.#...#...#...#.#.#...#...#.#...........#...#.#.#...#.....#.........#.#.#.....#.......#.....#.#...#...#.#.#...#.#.#.#...#.#...#.#.#.....#.#.#
#.#.#.#.#.#.###.#.###.#.#.#.#.###########.###.#.###.#.###.#########.#.#.#.#################.#.###.###.#.#.#.###.#.#.###.#.#.#.#.#.#####.#.#.#
#...#...#...###...###...#...#.............###...###...###...........#...#...................#.....###...#...###...#.....#...#...#.......#...#
#############################################################################################################################################"""